In [1]:
import re
import os, json
from dotenv import load_dotenv
from textwrap import dedent
from pprint import pprint
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")
import requests
response = requests.get("http://127.0.0.1:11434")
print(response.text)
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

Ollama is running


In [19]:
# 기본 유틸
import os
import json
import re
from pathlib import Path
from dotenv import load_dotenv
import warnings
warnings.filterwarnings("ignore")

# 문서 로딩 & 전처리
from langchain.document_loaders import Docx2txtLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

# 벡터화 (임베딩) / 검색
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# LLM 모델
from langchain.chat_models import ChatOpenAI
# (만약 Ollama 쓰면)
from langchain_ollama import ChatOllama

# 체인 및 에이전트
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.prompts.chat import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain.agents import initialize_agent, Tool
from langgraph.graph import StateGraph, START, END
from langchain.prompts import PromptTemplate

# 외부 API 호출 등 필요 시
import requests



In [20]:
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")
llm_model = ChatOllama(model="qwen3:1.7b",temperature=0.1)

In [10]:
# 2. 문서 로딩 및 분할
resume_path = Path("./data/tutorial-korean.pdf")  # sample_resume.pdf 또는 sample_resume.docx 경로 지정

# 로더 선택
if resume_path.suffix.lower() == ".pdf":
    loader = PyPDFLoader(str(resume_path))
else:
    loader = Docx2txtLoader(str(resume_path))

# 로딩 및 분할
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(docs)
print(f"분할된 청크 개수: {len(chunks)}")

# 3. 임베딩 및 벡터스토어 구축
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
vectorstore = FAISS.from_documents(chunks, embeddings)

# 4. RetrievalQA 체인 생성
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1)
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "아래는 지원자 이력서에서 추출된 정보입니다:\n"
        "{context}\n\n"
        "위 정보를 참고해, '{question}'에 대해 답변해 주세요."
    )
)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 4}),
    return_source_documents=False,
    chain_type_kwargs={"prompt": prompt}
)
# 5. 예시 질의
question = "백엔드 개발자 포지션에 지원할 때 강조하면 좋을 기술은 무엇일까?"
answer = qa_chain.run(question)
print("▶️ AI 답변:\n", answer)


분할된 청크 개수: 66
▶️ AI 답변:
 지원자는 디버깅에 대한 기본적인 지식과 BlueJ를 사용한 디버깅 경험을 갖고 있습니다. 이는 백엔드 개발자로서 중요한 기술 중 하나입니다. 디버깅은 코드 작성 중 발생하는 오류를 신속하게 해결하고 효율적인 코드를 작성하는 데 도움이 됩니다. 또한, 중단점 설정, 단계별 코드 실행, 변수 검사 등의 디버깅 기능을 활용하여 코드를 분석하고 수정하는 능력을 갖추고 있습니다.

따라서, 백엔드 개발자 포지션에 지원할 때는 이러한 디버깅 기술 및 경험을 강조하여 자신이 문제 해결능력을 갖추고 있다는 것을 강조할 수 있습니다. 또한, BlueJ를 사용한 경험을 통해 새로운 도구나 환경에 빠르게 적응할 수 있는 능력을 갖추고 있다는 것을 언급할 수 있습니다. 이는 백엔드 시스템을 개발하고 유지보수하는 과정에서 중요한 역할을 할 수 있는 기술이며, 지원자의 역량을 강조할 수 있는 요소가 될 것입니다.


In [11]:
analysis_prompt = PromptTemplate(
    input_variables=["text_input", "analysis_task"],
    template="""
당신은 전문 텍스트 분석가입니다.
아래 주어진 텍스트를 "{analysis_task}" 관점에서 자세히 분석해 주세요.

––––––––––––––––––––––––––––––
{text_input}
––––––––––––––––––––––––––––––

분석 내용:
"""
)
prompt = analysis_prompt.format(
    text_input="이력서: 홍길동, Java/Spring 3년, AWS 배포 경험 있음.",
    analysis_task="주요 강점과 보완해야 할 부분"
)
response = llm_model.predict(prompt)
print(response)

<think>
Okay, let's tackle this analysis. The user wants me to evaluate the resume entry from the perspective of "major strengths and areas for improvement." 

First, I need to identify the key points. The resume says "홍길동, Java/Spring 3년, AWS 배포 경험 있음." So, the person has 3 years of Java/Spring experience and some AWS deployment experience.

Strengths: The main strengths here are the technical skills in Java and Spring, which are fundamental in the tech industry. The AWS deployment experience is a plus, showing familiarity with cloud services. Also, the resume is concise, which is good for a professional profile.

Areas for Improvement: The resume is a bit generic. It doesn't mention specific projects or achievements, which would make it more compelling. The experience is listed as 3 years, but without details, it's hard to quantify the impact. Also, the AWS experience is mentioned but not elaborated on, so there's a gap in showing how they applied their skills. Additionally, the resu

In [14]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
chat_llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.1,
    streaming=False  # 메시지 리스트 호출 시에는 False 로 두는 게 일반적
)

analysis_chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(
        "당신은 AI 커리어 상담 분야의 전문가입니다. 사용자가 올린 텍스트를 아래 지시대로 분석해 주세요."
    ),
    HumanMessagePromptTemplate.from_template(
        "텍스트:\n{text_input}\n\n분석 관점: {analysis_task}"
    )
])
messages = analysis_chat_prompt.format_messages(
    text_input="이력서: 홍길동, Java/Spring 3년, AWS 배포 경험 있음.",
    analysis_task="지원 직무(백엔드)와의 적합도 평가"
)
response = chat_llm(messages)  # ChatOpenAI.streaming=False 등
print(response)


content='주어진 이력서를 백엔드 개발자로 평가해보겠습니다.\n\n1. **기술 스택**:\n   - Java와 Spring 프레임워크 경험이 있으므로 백엔드 개발에 필요한 기본적인 기술 스택을 보유하고 있습니다.\n   - AWS 배포 경험이 있어 클라우드 환경에서의 서비스 배포 및 관리에 대한 이해가 있을 것으로 예상됩니다.\n\n2. **경력**:\n   - 3년간의 Java와 Spring 경력을 보유하고 있어, 실무에서의 경험이 있을 것으로 판단됩니다.\n\n3. **부족한 점**:\n   - 특정 프로젝트나 업무 내용에 대한 구체적인 언급이 없어, 실제 업무에서의 성과나 역량에 대한 정보가 부족합니다.\n   - 다른 백엔드 기술에 대한 경험이나 지식 여부는 확인되지 않습니다.\n\n종합적으로, 이 지원자는 백엔드 개발자로서의 기본적인 기술 스택과 경험을 갖추고 있지만, 보다 구체적인 프로젝트나 역량에 대한 정보가 필요합니다. 추가적인 면접이나 심층적인 이력서 검토를 통해 능력을 더 정확히 평가할 필요가 있습니다.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 423, 'prompt_tokens': 120, 'total_tokens': 543, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run--e2f8eb82-c9ae-43da-926e-7fd673

### 이거보시면 됩니다.

In [27]:
import io
from pathlib import Path
from dotenv import load_dotenv
from langchain.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.chat_models import ChatOpenAI

load_dotenv()

# 1) 분석용 PromptTemplate 예시
analysis_template = PromptTemplate(
    input_variables=["text_input", "analysis_task"],
    template="""
당신은 커리어 상담 전문가입니다.
아래 텍스트를 "{analysis_task}" 관점에서 분석해 주세요.
한 문장 씩 줄바꿈해서 출력해주세요.

—— 원문 시작 ——
{text_input}
—— 원문 끝 ——

분석 결과:
"""
)

# 2) ChatPromptTemplate 예시
analysis_chat = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(
        "당신은 IT분야 커리어 상담 전문가입니다."
    ),
    HumanMessagePromptTemplate.from_template(
        "다음 텍스트를 \"{analysis_task}\" 관점에서 분석해 주세요:\n\n"
        "—————\n"
        "{text_input}\n"
        "—————"
    )
])

# 3) 파일 → 텍스트 추출 함수
def extract_text_from_file(file_path: Path) -> str:
    suffix = file_path.suffix.lower()
    if suffix == ".pdf":
        loader = PyPDFLoader(str(file_path))
    elif suffix in (".docx", ".doc"):
        loader = Docx2txtLoader(str(file_path))
    else:
        raise ValueError("PDF 또는 DOCX 파일만 지원합니다.")
    docs = loader.load()
    # 여러 페이지/청크를 하나의 문자열로 합치기
    return "\n\n".join([d.page_content for d in docs])

# 4) 최종 프롬프트 생성 함수 (PromptTemplate 버전)
def make_analysis_prompt(file_path: Path, analysis_task: str) -> str:
    text = extract_text_from_file(file_path)
    # 너무 길면 앞뒤만 자르고 중간 생략 처리 가능
    if len(text) > 3000:
        text = text[:1500] + "\n…(생략)…\n" + text[-1500:]
    return analysis_template.format(
        text_input=text,
        analysis_task=analysis_task
    )

# 5) Chat API 호출 예시
chat_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)

def run_analysis_via_chat(file_path: Path, analysis_task: str):
    text = extract_text_from_file(file_path)
    messages = analysis_chat.format_messages(
        text_input=text,
        analysis_task=analysis_task
    )
    return chat_llm(messages)

# ——— 사용 예 ———
from pathlib import Path

file_path = Path("./data/a.pdf")
task = "가지고 있는 기술 스택들과 커리어 분석"

# 1) 단순 문자열 프롬프트 생성
prompt_str = make_analysis_prompt(file_path, task)
print(prompt_str)

# 2) Chat API 호출
#response = run_analysis_via_chat(file_path, task)
#print(response)
response = qa_chain.run(question)   # 혹은 chat_llm(messages).content
print(response)



당신은 커리어 상담 전문가입니다.
아래 텍스트를 "가지고 있는 기술 스택들과 커리어 분석" 관점에서 분석해 주세요.
한 문장 씩 줄바꿈해서 출력해주세요.

—— 원문 시작 ——
John Doe
Email: john.doe@example.com | Phone: (123) 456-7890
LinkedIn: linkedin.com/in/johndoe
Professional Summary
Experienced Backend Developer with over 5 years of experience designing and
implementing scalable web applications using Java, Spring Boot, and AWS.
Passionate about system architecture, performance optimization, and clean code.
Experience
Senior Backend Developer, TechCorp Inc.
- Lead development of microservices architecture using Spring Boot and Docker.
- Implemented RESTful APIs consumed by front-end and mobile clients.
- Optimized database queries, reducing response time by 30%.
Backend Developer, WebSolutions LLC
- Developed features for e-commerce platform using Java and Spring MVC.
- Collaborated with DevOps to automate CI/CD pipelines on AWS.
- Wrote unit and integration tests achieving 85% code coverage.
Education
B.S. in Computer Science, University of Technology

### LangGraph 건드리는 중

In [10]:
# 1) 핸들러 정의
def parse_resume(state):
    path = state["resume_path"]
    if path.suffix.lower() == ".pdf":
        docs = PyPDFLoader(str(path)).load()
    else:
        docs = Docx2txtLoader(str(path)).load()
    state["chunks"] = RecursiveCharacterTextSplitter(1000,200).split_documents(docs)
    return state

def embed_documents(state):
    emb = OpenAIEmbeddings(model="text-embedding-ada-002")
    state["vectorstore"] = FAISS.from_documents(state["chunks"], emb)
    return state

def run_qa(state):
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)
    prompt = PromptTemplate(
        input_variables=["context","question"],
        template="이력서 정보:\n{context}\n질문: {question}"
    )
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=state["vectorstore"].as_retriever(k=4),
        chain_type="stuff",
        chain_type_kwargs={"prompt": prompt}
    )
    state["answer"] = qa_chain.run(state["question"])
    return state

# 2) 그래프 구성
from typing_extensions import TypedDict

# 상태 스키마 정의
class RAGState(TypedDict):
    resume_path: Path
    question: str
    chunks: list[Document]
    vectorstore: FAISS
    answer: str

# 2) 스키마를 넘겨서 그래프 생성
graph = StateGraph(state_schema=RAGState)

# 3) 노드 추가 (핸들러는 state를 RAGState로 받아 리턴)
graph.add_node(START)
graph.add_node("parse", handler=parse_resume)
graph.add_node("embed", handler=embed_documents)
graph.add_node("qa", handler=run_qa)
graph.add_node(END)

graph.add_edge(START, "parse")
graph.add_edge("parse", "embed")
graph.add_edge("embed", "qa")
graph.add_edge("qa", END)

# 4) 실행
initial_state: RAGState = {
    "resume_path": Path("sample_resume.pdf"),
    "question": "백엔드 지원 시 강조할 기술은?"
}
final_state = graph.run(initial_state)
print("AI 답변:", final_state["answer"])

graph.add_node(START)
graph.add_node("parse", handler=parse_resume)
graph.add_node("embed", handler=embed_documents)
graph.add_node("qa", handler=run_qa)
graph.add_node(END)

graph.add_edge(START, "parse")
graph.add_edge("parse", "embed")
graph.add_edge("embed", "qa")
graph.add_edge("qa", END)

# 3) 실행
initial_state = {
    "resume_path": Path("sample_resume.pdf"),
    "question": "백엔드 개발자 지원 시 강조할 기술은?"
}
final_state = graph.run(initial_state)
print("AI 답변:", final_state["answer"])


RuntimeError: 